In [ ]:
import tqdm
import numpy as np
import torch
import pandas as pd
import pickle
import time
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.set_option('mode.chained_assignment',  None)
import os

import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import random
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader


In [ ]:
from sklearn.decomposition import PCA
from umap import UMAP

In [ ]:
with open('TF_in_data_all.pickle', 'rb') as f:
    TF_in_data, TF_in_std, TF_in_mask, ID_list, ID_year_list = pickle.load(f)
TF_in_data = TF_in_data.detach().cpu()
TF_in_std = TF_in_std.detach().cpu()
TF_in_mask = TF_in_mask.detach().cpu()

In [ ]:
loss_dir = './loss'
model_dir = './model_save'
table_loc_master = "/data/Storage_DAS02/jaeyoon/230101_BreastCancer_Project/data/230629_BC_new/"
table_data_master = "230913_BT_Timeseries_new.xlsx"

Table_1 = pd.read_excel(table_loc_master+table_data_master).reset_index(drop = True)
# Table_Date = Table_1[["Test_date_normed"]]
Table_Y_date = Table_1[["Date"]]
Table_Y_date.index = [Table_1["new_ID"],Table_1["count_in_TF"]]
Table_Y_date["Date"] = [datetime.datetime.strptime(i, '%Y-%m-%d').date() for i in Table_Y_date["Date"]]
Table_Y_date

In [ ]:
ID_test = pd.read_excel("Test_ID_set.xlsx").values[:,1]
ID_train = pd.read_excel("Train_ID_set.xlsx").values[:,1]
# ID_test = np.unique([i.split(',')[0][2:-1] for i in ID_test])
# ID_train = np.unique([i.split(',')[0][2:-1] for i in ID_train])
len(ID_test)

In [ ]:
data_loc = "/data/Storage_DAS02/jaeyoon/230101_BreastCancer_Project/data/230629_BC_new"
data_recur = "Recur_new_format_all.xlsx"
Table_Y = pd.read_excel(os.path.join(data_loc,data_recur)).set_index(['new_ID'])
Table_Recur = pd.DataFrame(columns = ["Op_date", "Recur_OX", "last_record", "record_before_recur", "recur_date"])
for ID in np.unique(Table_Y.index):
    if ID in np.append(ID_test,ID_train):
        try:
            Recur_OX = Table_Y.loc[ID]["Recur"].values[0] != pd.Timestamp('2100-01-01 00:00:00')
        except:
            Recur_OX = Table_Y.loc[ID]["Recur"] != pd.Timestamp('2100-01-01 00:00:00')
        try:
            Op_date = Table_Y.loc[ID]["Op"].values[0]
            last_record = Table_Y.loc[ID]["Date"].values[-1]
        except:
            Op_date = Table_Y.loc[ID]["Op"]
            last_record = Table_Y.loc[ID]["Date"]
        
        if Recur_OX == 0:
            Table_Recur.loc[ID] = [Op_date, Recur_OX, last_record, pd.Timestamp('2100-01-01 00:00:00'), pd.Timestamp('2100-01-01 00:00:00')]
        elif Recur_OX == 1:
            try:
                Recur_date = Table_Y.loc[ID]["Recur"].values[0]
            except:
                Recur_date = Table_Y.loc[ID]["Recur"]
            record_before_recur = Table_Y.loc[ID]["Date"].values[Table_Y.loc[ID]["Date"].values<Recur_date][-1]
            Table_Recur.loc[ID] = [Op_date, Recur_OX, last_record, record_before_recur, Recur_date]

In [ ]:
Table_Recur_now = pd.DataFrame(columns= [i for i in range(10)])
ii = 0
for ID in np.append(ID_test,ID_train):
    Table_Recur_now.loc[ID] = np.nan
    if (ID in Table_Y_date.index) & (ID in Table_Recur.index):
        ii+=1
        Recur_info = Table_Recur.loc[ID]
        
        if Recur_info["Recur_OX"]:
            Table_Recur_now.loc[ID][[ii for ii in range(min(10,max(1,max([1]+[j+1 for i,j in zip(Table_Y_date.loc[ID].values,Table_Y_date.loc[ID].index) if (i <= Recur_info['record_before_recur'].date())&(type(j)!=str) ])+1)))]] = 0
            Table_Recur_now.loc[ID][[ii for ii in range(min(9,min([9]+[j+1 for i,j in zip(Table_Y_date.loc[ID].values,Table_Y_date.loc[ID].index) if (i > Recur_info['recur_date'].date())&(type(j)!=str) ])-1),10)]] = 1
        else:
            Table_Recur_now.loc[ID][[ii for ii in range(min(10,max(1,max([1]+[j+1 for i,j in zip(Table_Y_date.loc[ID].values,Table_Y_date.loc[ID].index) if (i <= Recur_info['last_record'].date())&(type(j)!=str) ])+1)))]] = 0

In [ ]:
data_drug = "230903_Table_Drug.xlsx"
Table_D = pd.read_excel(os.path.join(data_loc,data_drug)).set_index(['ID','year'])
Table_D = (Table_D>0)+0
Table_D

In [ ]:
# DataLoader
class BART_predict_Dataset(Dataset):
    def __init__(self, inputs):
        TF_in_data, TF_in_mask, ID_list, ID_year_list, Table_Recur_sub, Table_D, ID_use = inputs
        idx_sub = [n for n,i in enumerate(ID_list) if i in ID_use]
        ID_use = [i for n,i in enumerate(ID_list) if i in ID_use]
        self.data = TF_in_data[idx_sub] # N, 10, 50
        self.mask = TF_in_mask[idx_sub] # N, 10, 1
        self.ID_list = np.array(ID_list)[idx_sub]
        self.ID_year_list = [i for n,i in enumerate(ID_year_list) if n in idx_sub]
        
        self.Recur_mask_TF = torch.from_numpy(Table_Recur_sub.loc[ID_use].notna().values) # N, 10
        self.Recur_data_TF = torch.from_numpy(Table_Recur_sub.loc[ID_use].fillna(0).values) # N, 10
        
        self.D_data_TF = torch.zeros((TF_in_data.shape[0],10,Table_D.shape[1]))
        D_idx = Table_D.index.get_level_values('ID').unique()
        for n,ID in enumerate(ID_use):
            if ID in D_idx:
                D_sub = Table_D.loc[ID]
                for y_idx in D_sub.index:
                    self.D_data_TF[n,y_idx,:] = torch.from_numpy(D_sub.loc[y_idx].values.astype(float))
            
    def __len__(self):
        return len(self.ID_list)

    def __getitem__(self, idx_sub): # N x 260x512
        # 260 x 512
        return {'data' : self.data[idx_sub],
                'mask' : self.mask[idx_sub],
                'idx' : self.ID_list[idx_sub],
                'Recur_data' : self.Recur_data_TF[idx_sub],
                'Recur_mask' : self.Recur_mask_TF[idx_sub],
                'Drug_data' : self.D_data_TF[idx_sub],
                'data_idx' : self.ID_year_list[idx_sub]}
        
 
class BART_sampler():
    def __init__(self, rand_p = 0.05):
        self.rand_p = rand_p
    def BART_rand_fn(self, samples):
        # x1,x2,m1,m2,op_data = zip(*batch)
        # r_mask_ratio = self.rand_p
        # eps = 1e-6
        
        # x1 = torch.stack(x1).float()
        # x2 = torch.stack(x2).float()
        # m1 = torch.stack(m1).bool()
        # m2 = torch.stack(m2).bool()
        # op_data = torch.stack(op_data).float()
        
        p = self.rand_p
        data = torch.stack([sample['data'] for sample in samples]) # N, 10, 50
        mask = torch.stack([sample['mask'] for sample in samples]).bool() # N, 10, 1
        Recur_data = torch.stack([sample['Recur_data'] for sample in samples]) # N, 10
        Recur_mask = torch.stack([sample['Recur_mask'] for sample in samples]).bool() # N, 10 (value_O -> 1)
        Recur_data = torch.concat([torch.zeros(Recur_data.shape[0],1), Recur_data],axis = -1) # -> N, 11
        Recur_mask = torch.concat([torch.ones(Recur_mask.shape[0],1), Recur_mask],axis = -1).bool() # -> N, 11
        Drug_data = torch.stack([sample['Drug_data'] for sample in samples])
        IDs = [sample['idx'] for sample in samples] # N, 10, 1
        Rand_mask = torch.zeros_like(mask)
        if p > 0:
            for n,sample in enumerate(samples):
                idx_sub = np.setdiff1d(sample['data_idx'],[1])
                if idx_sub.shape[0] != 0:
                    idx_MLM = np.random.choice(idx_sub,max(1,int(len(idx_sub) * p)),replace = False)            
                    for idx in idx_MLM:
                        Rand_mask[n,idx] = 1
        Rand_mask = Rand_mask.bool()
        Enc_mask = mask + Rand_mask
        # Enc_mask = mask
        Enc_in = data * (~Enc_mask)
        Dec_in = torch.concat([torch.zeros(data.shape[0],1,data.shape[2]).float(),Enc_in[:,:-1,:]], axis = 1)
        Dec_mask = torch.concat([torch.zeros(data.shape[0],1,1),Enc_mask[:,:-1,:]], axis = 1).repeat(1,1,10).bool()
        
        return Enc_in, Enc_mask, Dec_in, Dec_mask, Recur_data.float(), Recur_mask, Drug_data, IDs    

In [ ]:
ID_train = np.setdiff1d(ID_train,np.array(ID_list)[(10-TF_in_mask.sum(axis = [1,2]) == 1)])
ID_test = np.setdiff1d(ID_test,np.array(ID_list)[(10-TF_in_mask.sum(axis = [1,2]) == 1)])
print(len(ID_train),len(ID_test))

In [ ]:
ID_recur = Table_Recur_now.index[Table_Recur_now.max(axis = 1) == 1]
ID_normal = Table_Recur_now.index[Table_Recur_now.max(axis = 1) == 0]


In [ ]:
batch_size = 64
BART_sampler_use = BART_sampler(0.2)
recur_model_dataloader = DataLoader(BART_predict_Dataset([TF_in_data, TF_in_mask, ID_list, ID_year_list, Table_Recur_now, Table_D, ID_recur]),
                            batch_size=batch_size, shuffle=True, collate_fn=BART_sampler_use.BART_rand_fn)
normal_model_dataloader = DataLoader(BART_predict_Dataset([TF_in_data, TF_in_mask, ID_list, ID_year_list, Table_Recur_now, Table_D, ID_normal]),
                            batch_size=batch_size, shuffle=True, collate_fn=BART_sampler_use.BART_rand_fn)

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio,bias = True):
        super().__init__()

        assert hidden_dim % n_heads == 0
        self.hidden_dim = hidden_dim # 임베딩 차원
        self.n_heads = n_heads # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
        self.head_dim = hidden_dim // n_heads # 각 헤드(head)에서의 임베딩 차원
        self.fc_q = nn.Linear(hidden_dim, hidden_dim,bias = bias) # Query 값에 적용될 FC 레이어
        self.fc_k = nn.Linear(hidden_dim, hidden_dim,bias = bias) # Key 값에 적용될 FC 레이어
        self.fc_v = nn.Linear(hidden_dim, hidden_dim,bias = bias) # Value 값에 적용될 FC 레이어

        self.fc_o = nn.Linear(hidden_dim, hidden_dim,bias = bias)

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # hidden_dim → n_heads X head_dim 형태로 변형
        # n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)


        # Attention Energy 계산
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / math.sqrt(self.hidden_dim)
        energy = torch.clamp(energy,max = 1e6, min = -1e6)
        # 마스크(mask)를 사용하는 경우
        if mask is not None:
            energy = energy.masked_fill(mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1), -1e10)
        attention = torch.softmax(energy, dim=-1)
        
        x = torch.matmul(self.dropout(attention), V)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(batch_size, -1, self.hidden_dim)
        x = self.fc_o(x)

        return x, attention
    
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hidden_dim, pf_dim, dropout_ratio, bias = True):
        super().__init__()

        self.fc_1 = nn.Linear(hidden_dim, pf_dim,bias = bias)
        self.fc_2 = nn.Linear(pf_dim, hidden_dim,bias = bias)

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):

        x = self.dropout(torch.relu(self.fc_1(x)))
        x = self.fc_2(x)

        return x
    
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 하나의 임베딩이 복제되어 Query, Key, Value로 입력되는 방식
    def forward(self, src, src_mask):

        # self attention
        _src, attn = self.self_attention(src, src, src, src_mask)
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        # position-wise feedforward
        _src = self.positionwise_feedforward(src)
        src = self.ff_layer_norm(src + self.dropout(_src))

        return src, attn
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.tok_embedding = nn.Linear(input_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(input_dim, hidden_dim)

        self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio) for _ in range(n_layers)])
        self.dropout = nn.Dropout(dropout_ratio)


    def forward(self, src, src_mask):

        batch_size = src.shape[0]
        src_len = src.shape[1]
        device = src.device
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(device)

        self.tok_embedding.to(device)
        self.pos_embedding.to(device)
        # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
        src = self.dropout((self.tok_embedding(src) * math.sqrt(self.hidden_dim)) + self.pos_embedding(pos))
        
        # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
        attns = []
        for layer in self.layers:
            src, attn = layer(src, src_mask)
            attns += [attn]

        return src, attns # 마지막 레이어의 출력을 반환
    
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio)
        self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # self attention
        _trg, attn_1 = self.self_attention(trg, trg, trg, trg_mask)
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        # encoder attention
        _trg, attn_2 = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        return trg, attn_1, attn_2
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.tok_embedding = nn.Linear(output_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(output_dim, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio) for _ in range(n_layers)])

        self.fc_out_1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_out_2 = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout_ratio)


    def forward(self, trg, enc_src, trg_mask, src_mask):

        device = trg.device
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(device)

        trg = self.dropout((self.tok_embedding(trg) * math.sqrt(self.hidden_dim)) + self.pos_embedding(pos))

        # trg: [batch_size, trg_len, hidden_dim]
        attns_1 = []
        attns_2 = []
        for layer in self.layers:
            trg, attn_1, attn_2 = layer(trg, enc_src, trg_mask, src_mask)
            attns_1 += [attn_1]
            attns_2 += [attn_2]

        trg = self.fc_out_1(trg)
        output = self.fc_out_2(trg)

        return output, attns_1, attns_2
    
class Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.trg_sub_mask = torch.tril(torch.ones((10, 10)),diagonal=-1).bool().T.unsqueeze(0)
    
    def forward(self, inputs):
        Enc_in, Enc_mask, Dec_in, Dec_mask = inputs
        
        device = Enc_in.device
        Dec_mask = Dec_mask.transpose(1,2) + self.trg_sub_mask.to(device)
        Enc_mask = Enc_mask.transpose(1,2)

        enc_src, enc_attns = self.encoder(Enc_in, Enc_mask)

        output, dec_attns_1, dec_attns_2 = self.decoder(Dec_in, enc_src, Dec_mask, Enc_mask)

        return output, enc_attns + dec_attns_1 + dec_attns_2

In [ ]:
class DecoderLayer_pred(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio,bias = False)
        self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio,bias = False)
        self.drug_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio,bias = False)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio,bias = False)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask, drug_data, drug_mask, with_drug):
        # self attention
        _trg, attn_1 = self.self_attention(trg, trg, trg, trg_mask)
        if with_drug != 0:
            _trg_2, attn_2 = self.drug_attention(trg, drug_data, drug_data, drug_mask)
            trg = self.self_attn_layer_norm(trg + self.dropout(_trg)+ self.dropout(_trg_2))
        else:
            trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            attn_2 = torch.zeros_like(attn_1)
        # trg: [batch_size, trg_len, hidden_dim]

        # encoder attention
        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        _trg_1, attn_3 = self.encoder_attention(trg, enc_src, enc_src, src_mask)

        # dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg_1))

        # trg: [batch_size, trg_len, hidden_dim]

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        # dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return trg, attn_1, attn_2, attn_3
    
class Decoder_pred(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, output_dim = 0):
        super().__init__()
        if output_dim == 0:
            output_dim = input_dim
        self.hidden_dim = hidden_dim
        self.tok_embedding = nn.Linear(input_dim, hidden_dim,bias = False)
        self.drug_embedding = nn.Linear(25, hidden_dim,bias = False)
        self.pos_embedding = nn.Embedding(hidden_dim, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer_pred(hidden_dim, n_heads, pf_dim, dropout_ratio) for _ in range(n_layers)])

        self.fc_out_1 = nn.Linear(hidden_dim*10, hidden_dim,bias = False)
        self.fc_out_2 = nn.Linear(hidden_dim, output_dim,bias = False)

        self.dropout = nn.Dropout(dropout_ratio)


    def forward(self, trg, enc_src, trg_mask, src_mask,drug_data= 0, drug_mask= 0, with_drug=0):

        # trg: [batch_size, trg_len]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]
        device = trg.device
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(device)

        # pos: [batch_size, trg_len]
        trg = self.dropout((self.tok_embedding(trg) * math.sqrt(self.hidden_dim)) + self.pos_embedding(pos))
        if with_drug != 0:
            drug_data = self.drug_embedding(drug_data)
        # trg: [batch_size, trg_len, hidden_dim]
        attns_1 = []
        attns_2 = []
        attns_3 = []
        trgs = [trg]
        for layer in self.layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            trg, attn_1, attn_2, attn_3 = layer(trg, enc_src, trg_mask, src_mask, drug_data, drug_mask, with_drug = with_drug)
            attns_1 += [attn_1]
            attns_2 += [attn_2]
            attns_3 += [attn_3]
            trgs += [trg]
        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]
        trg = trg.reshape(trg.shape[0],-1)
        trg = nn.GELU()(self.fc_out_1(trg))
        output = self.fc_out_2(trg)

        # output: [batch_size, trg_len, output_dim]

        return output, attns_1, attns_2, attns_3, trgs, trg

In [ ]:
class Transformer_pred(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.trg_sub_mask = torch.tril(torch.ones((10, 10)),diagonal=-1).bool().T.unsqueeze(0)
    
    def forward(self, inputs, with_drug = True):
        Enc_in, Enc_mask, Dec_in, Dec_mask, drug_data = inputs
        device = Enc_in.device
        # Dec_mask = Enc_mask.transpose(1,2) + self.trg_sub_mask.to(device)
        Dec_mask = Dec_mask.transpose(1,2) + self.trg_sub_mask.to(device)
        Enc_mask = Enc_mask.transpose(1,2)
        drug_mask = torch.zeros_like(Dec_mask) + self.trg_sub_mask.to(device)
        # Dec_in = torch.zeros_like(Dec_in)
        Dec_in = Enc_in.clone()
        enc_src, enc_attns = self.encoder(Enc_in, Enc_mask)

        if with_drug:
            output, dec_attns_1, dec_attns_2, dec_attns_3, trgs, trg = self.decoder(Dec_in, enc_src, Dec_mask, Enc_mask, drug_data, drug_mask, with_drug = with_drug)
        else:
            output, dec_attns_1, dec_attns_2, dec_attns_3, trgs, trg = self.decoder(Dec_in, enc_src, Dec_mask, Enc_mask, with_drug = with_drug)
            
        return output, enc_attns + dec_attns_1 + dec_attns_2 + dec_attns_3, trgs, trg

In [ ]:
devices = [torch.device(f"cuda:{i}") for i in range(2, torch.cuda.device_count()-1)]
device = devices[0]
with open(f"./model_save/230913_pretrained_BART.pth","rb") as fw:
    BART_model = torch.load(fw, map_location=device)


In [ ]:
input_dim, hidden_dim, layer_num, heads, ff_dim, dropout_ratio = 100, 512, 1, 8, 1024, 0
Trained_Encoder = BART_model.module.encoder
for para in Trained_Encoder.parameters():
    para.requires_grad = False
output_dim = 128
dec_recur = Decoder_pred(input_dim, hidden_dim, layer_num, heads, ff_dim, dropout_ratio, output_dim)
dec_normal = Decoder_pred(input_dim, hidden_dim, layer_num, heads, ff_dim, dropout_ratio, output_dim)
OneClass_recur_model = Transformer_pred(Trained_Encoder, dec_recur).to(device)
OneClass_normal_model = Transformer_pred(Trained_Encoder, dec_normal).to(device)
# optimizer = torch.optim.AdamW(Pred_BART_model.parameters(), lr=1e-6)
optimizer_recur = torch.optim.RAdam(OneClass_recur_model.parameters(), lr=1e-4)
optimizer_normal = torch.optim.RAdam(OneClass_normal_model.parameters(), lr=1e-4)

In [ ]:
point_recur = torch.rand(output_dim).to(device)
point_normal = torch.rand(output_dim).to(device)
point_recur = (point_recur/(point_recur.pow(2).sum().sqrt())).unsqueeze(0)*10
point_normal = (point_normal/(point_normal.pow(2).sum().sqrt())).unsqueeze(0)*10
R_len = 1
(point_recur-point_normal).pow(2).sum().sqrt()

In [ ]:
torch.save(Pred_BART_model, f"{model_dir}/{save_name}.pth")

In [ ]:
n_epochs = 300
save_name = f"230914_BART_Oneclass_01"
tqdm_epoch = tqdm.tqdm(range(n_epochs))
loss_train_all_list = []
loss_test_all_list = []
# year_weight = torch.tensor([[np.log2(i) for i in range(1,11)]]).float().to(device)
for epoch in tqdm_epoch:
    epoch_save = ((epoch+1) % 10 == 0)
    loss_train_all = 0
    loss_test_all = 0
    OneClass_recur_model.train()
    len_train = 0
    for Enc_in, Enc_mask, Dec_in, Dec_mask,  Recur_data, Recur_mask, drug_data, IDs in recur_model_dataloader:
        len_train += Enc_in.shape[0]
        Enc_in = Enc_in.to(device) # N,10,26
        Dec_in = Dec_in.to(device) # N,10,26
        Enc_mask = Enc_mask.to(device) # N,10,1
        Dec_mask = Dec_mask.to(device) # N,10,1
        Recur_data= Recur_data.to(device)
        Recur_mask= Recur_mask.to(device)
        drug_data = drug_data.to(device)
        A,_,_,_ = OneClass_recur_model([Enc_in, Enc_mask, Dec_in, Dec_mask, drug_data], with_drug  = True)
        # loss = ((Recur_mask[:,1:] == 1)*(torch.sigmoid(A.squeeze()) - Recur_data[:,1:])).pow(2).sum()/((Recur_mask[:,1:] == 1).sum())
        loss = torch.max((A-point_recur).pow(2).sum(axis = 1)-R_len,torch.zeros(A.shape[0]).to(device)).mean()
        # loss = ((Recur_mask == 1)*(torch.sigmoid(A.squeeze()) - Recur_data)).pow(2).sum()/((Recur_mask == 1).sum())
        optimizer_recur.zero_grad()
        loss.backward()
        optimizer_recur.step()
        loss_train_all += loss.item()*Enc_in.shape[0]
    loss_train_all /= len_train
    # if epoch_save: # Save Test as file
    #     torch.save(Pred_BART_model, f"{model_dir}/{save_name}.pth")
    discript = f"model Oneclass | "
    discript_dict = {}
    discript += f"Loss :"
    discript += f"Train : {loss_train_all:.3f} # {len_train}"
    tqdm_epoch.set_description(discript)
    loss_train_all_list += [loss_train_all]

In [ ]:
n_epochs = 100
save_name = f"230914_BART_Oneclass_01"
tqdm_epoch = tqdm.tqdm(range(n_epochs))
# loss_train_all_list = []
# loss_test_all_list = []
# year_weight = torch.tensor([[np.log2(i) for i in range(1,11)]]).float().to(device)
for epoch in tqdm_epoch:
    epoch_save = ((epoch+1) % 10 == 0)
    loss_train_all = 0
    loss_test_all = 0
    OneClass_normal_model.train()
    len_test = 0
    for Enc_in, Enc_mask, Dec_in, Dec_mask, Recur_data, Recur_mask, drug_data, IDs in normal_model_dataloader:
        len_test += Enc_in.shape[0]
        Enc_in = Enc_in.to(device) # N,10,26
        Dec_in = Dec_in.to(device) # N,10,26
        Enc_mask = Enc_mask.to(device) # N,10,1
        Dec_mask = Dec_mask.to(device) # N,10,1
        Recur_data= Recur_data.to(device)
        Recur_mask= Recur_mask.to(device)
        drug_data= drug_data.to(device)
        A,_,_,_ = OneClass_normal_model([Enc_in, Enc_mask, Dec_in, Dec_mask, drug_data], with_drug  = True)
        # loss = ((Recur_mask[:,1:] == 1)*(torch.sigmoid(A.squeeze()) - Recur_data[:,1:])).pow(2).sum()/((Recur_mask[:,1:] == 1).sum())
        loss = torch.max((A-point_normal).pow(2).sum(axis = 1)-R_len,torch.zeros(A.shape[0]).to(device)).mean()
        optimizer_normal.zero_grad()
        loss.backward()
        optimizer_normal.step()
        loss_test_all += loss.item()*Enc_in.shape[0]
    loss_test_all /= len_test
    discript = f"model Oneclass | "
    discript_dict = {}
    discript += f"Loss :"
    discript += f"Normal : {loss_test_all:.3f} # {len_test}"
    tqdm_epoch.set_description(discript)
    loss_test_all_list += [loss_test_all]

In [ ]:
(A-point_normal).pow(2).sum(axis = 1)

In [ ]:
plt.plot(loss_train_all_list[-140:],'b')
plt.plot(loss_test_all_list[-140:],'r')
plt.legend(['train','validate'])
plt.xlabel('epoch (#)')
plt.ylabel('Loss (BCE)')
plt.title('Loss')
plt.show()

In [ ]:
OneClass_recur_model.eval()
OneClass_normal_model.eval()
Result_recurs = []
Result_normals = []
for Enc_in, Enc_mask, Dec_in, Dec_mask,  Recur_data, Recur_mask, drug_data, IDs in recur_model_dataloader:
    len_train += Enc_in.shape[0]
    Enc_in = Enc_in.to(device) # N,10,26
    Dec_in = Dec_in.to(device) # N,10,26
    Enc_mask = Enc_mask.to(device) # N,10,1
    Dec_mask = Dec_mask.to(device) # N,10,1
    Recur_data= Recur_data.to(device)
    Recur_mask= Recur_mask.to(device)
    drug_data = drug_data.to(device)
    A,_,_,_ = OneClass_recur_model([Enc_in, Enc_mask, Dec_in, Dec_mask, drug_data], with_drug  = True)
    B,_,_,_ = OneClass_normal_model([Enc_in, Enc_mask, Dec_in, Dec_mask, drug_data], with_drug  = True)
    # loss = ((Recur_mask[:,1:] == 1)*(torch.sigmoid(A.squeeze()) - Recur_data[:,1:])).pow(2).sum()/((Recur_mask[:,1:] == 1).sum())
    # Result_recur = torch.max((A-point_recur).pow(2).sum(axis = 1)-R_len,torch.zeros(A.shape[0]).to(device))
    # Result_normal = torch.max((B-point_normal).pow(2).sum(axis = 1)-R_len,torch.zeros(B.shape[0]).to(device))
    Result_recur = (A-point_recur).pow(2).sum(axis = 1)
    Result_normal = (B-point_normal).pow(2).sum(axis = 1)
    Result_recurs = Result_recur.detach().cpu().numpy() if len(Result_recurs) == 0 else np.concatenate([Result_recurs,Result_recur.detach().cpu().numpy()],axis = 0)
    Result_normals = Result_normal.detach().cpu().numpy() if len(Result_normals) == 0 else np.concatenate([Result_normals,Result_normal.detach().cpu().numpy()],axis = 0)
        

In [ ]:
Result_recurs2 = []
Result_normals2 = []
for Enc_in, Enc_mask, Dec_in, Dec_mask,  Recur_data, Recur_mask, drug_data, IDs in normal_model_dataloader:
    len_train += Enc_in.shape[0]
    Enc_in = Enc_in.to(device) # N,10,26
    Dec_in = Dec_in.to(device) # N,10,26
    Enc_mask = Enc_mask.to(device) # N,10,1
    Dec_mask = Dec_mask.to(device) # N,10,1
    Recur_data= Recur_data.to(device)
    Recur_mask= Recur_mask.to(device)
    drug_data = drug_data.to(device)
    A,_,_,_ = OneClass_recur_model([Enc_in, Enc_mask, Dec_in, Dec_mask, drug_data], with_drug  = True)
    B,_,_,_ = OneClass_normal_model([Enc_in, Enc_mask, Dec_in, Dec_mask, drug_data], with_drug  = True)
    # loss = ((Recur_mask[:,1:] == 1)*(torch.sigmoid(A.squeeze()) - Recur_data[:,1:])).pow(2).sum()/((Recur_mask[:,1:] == 1).sum())
    # Result_recur = torch.max((A-point_recur).pow(2).sum(axis = 1)-R_len,torch.zeros(A.shape[0]).to(device))
    # Result_normal = torch.max((B-point_normal).pow(2).sum(axis = 1)-R_len,torch.zeros(B.shape[0]).to(device))
    Result_recur = (A-point_recur).pow(2).sum(axis = 1)
    Result_normal = (B-point_normal).pow(2).sum(axis = 1)
    Result_recurs2 = Result_recur.detach().cpu().numpy() if len(Result_recurs2) == 0 else np.concatenate([Result_recurs2,Result_recur.detach().cpu().numpy()],axis = 0)
    Result_normals2 = Result_normal.detach().cpu().numpy() if len(Result_normals2) == 0 else np.concatenate([Result_normals2,Result_normal.detach().cpu().numpy()],axis = 0)
        

In [ ]:
plt.hist(Result_normals2)

In [ ]:
(Result_recurs2-1)[:10]

In [ ]:
((Result_recurs2-1)*(Result_recurs2>1))[:10]

In [ ]:
plt.hist(np.exp((-1)*(Result_recurs-1)*(Result_recurs>1))/(np.exp((-1)*(Result_recurs-1)*(Result_recurs>1))+np.exp((-1)*(Result_normals-1)*(Result_normals>1))),alpha = 0.2)
plt.show()
plt.hist(np.exp((-1)*(Result_recurs2-1)*(Result_recurs2>1))/(np.exp((-1)*(Result_recurs2-1)*(Result_recurs2>1))+np.exp((-1)*(Result_normals2-1)*(Result_normals2>1))),alpha = 0.2)

In [ ]:
plt.figure(figsize=(12,3))
plt.subplot(2,1,1)
plt.hist(np.exp((-1)*(Result_recurs))/(np.exp((-1)*(Result_recurs))+np.exp((-1)*(Result_normals))), color = 'r', bins = [i/20 for i in range(21)])
plt.title("Recurred patients")
plt.xticks([])
plt.xlim(0,1)
plt.subplot(2,1,2)
plt.hist(np.exp((-1)*(Result_recurs2))/(np.exp((-1)*(Result_recurs2))+np.exp((-1)*(Result_normals2))), color = 'b', bins = [i/20 for i in range(21)])
plt.title("Non-recurred patients")
plt.xlim(0,1)


In [ ]:

plt.scatter(np.exp((-1)*(Result_recurs)),np.exp((-1)*(Result_normals)), s = 1,c='r', alpha = 0.5)
plt.scatter(np.exp((-1)*(Result_recurs2)),np.exp((-1)*(Result_normals2)), s = 1,c='b', alpha = 0.5)
# plt.xlim(0,2.1)
# plt.ylim(0,2.1)
plt.xlabel('recur predict')
plt.ylabel('normal predict')
plt.legend(["Recur","Normal"])
plt.xlim(-0.1,1.1)
plt.ylim(-0.1,1.1)
plt.grid(1)

In [ ]:

plt.scatter(np.exp((-1)*(Result_recurs-1)*(Result_recurs>1)),np.exp((-1)*(Result_normals-1)*(Result_normals>1)), s = 1,c='r', alpha = 0.5)
plt.scatter(np.exp((-1)*(Result_recurs2-1)*(Result_recurs2>1)),np.exp((-1)*(Result_normals2-1)*(Result_normals2>1)), s = 1,c='b', alpha = 0.5)
# plt.xlim(0,2.1)
# plt.ylim(0,2.1)
plt.xlabel('recur predict')
plt.ylabel('normal predict')
plt.legend(["Recur","Normal"])
plt.grid(1)

In [ ]:
plt.scatter(np.log(1/Result_recurs),np.log(1/Result_normals), s = 1,c='r')
plt.scatter(np.log(1/Result_recurs2),np.log(1/Result_normals2), s = 1,c='b')

In [ ]:
# Loss :Train : 0.235 # 4103 Validate : 0.329
# Loss :Train : 0.211 # 4103 Validate : 0.261

In [ ]:
Pred_BART_model.eval()
A_all_val = []
a_all_val = []
b_all_val = []
c_all_val = []
d_all_val = []
B_all_val = []
D_all_val = []
# C_all_val = []
out_all_val = []
out_mas_val = []
ID_all = []
BART_sampler_use.rand_p = 0
for Enc_in, Enc_mask, Dec_in, Dec_mask, Recur_data, Recur_mask, drug_data, IDs in train_dataloader:
    len_test += Enc_in.shape[0]
    Enc_in = Enc_in.to(device) # N,10,26
    Dec_in = Dec_in.to(device) # N,10,26
    Enc_mask = Enc_mask.to(device) # N,10,1
    Dec_mask = Dec_mask.to(device) # N,10,1
    Recur_data= Recur_data.to(device)
    Recur_mask= Recur_mask.to(device)
    drug_data= drug_data.to(device)
    A,B,C,D = Pred_BART_model([Enc_in, Enc_mask, Enc_in, Enc_mask, drug_data], with_drug = True)
    # A,B = Pred_BART_model([Enc_in, Enc_mask, Recur_data[:,:-1].unsqueeze(2), ~Recur_mask[:,:-1].unsqueeze(2)])
    A = torch.sigmoid(torch.clamp(A.squeeze(),max = 1e6, min = -1e6))
    ID_all += IDs
    # B_all_val = B.detach().cpu().numpy() if len(B_all_val) == 0 else np.concatenate([B_all_val,B.detach().cpu().numpy()],axis = 0)
    D_all_val = D.detach().cpu().numpy() if len(D_all_val) == 0 else np.concatenate([D_all_val,D.detach().cpu().numpy()],axis = 0)
    # B_all_val = [i.detach().cpu().numpy() for i in B] if len(B_all_val) == 0 else [np.concatenate([i,j.detach().cpu().numpy()],axis = 0) for i,j in zip(B_all_val,B)]
    A_all_val = A.detach().cpu().numpy() if len(A_all_val) == 0 else np.concatenate([A_all_val,A.detach().cpu().numpy()],axis = 0)
    # C_all_val = [i.detach().cpu().numpy() for i in C] if len(C_all_val) == 0 else [np.concatenate([i,j.detach().cpu().numpy()],axis = 0) for i,j in zip(C_all_val,C)]
    a_all_val = Enc_in.detach().cpu().numpy() if len(a_all_val) == 0 else np.concatenate([a_all_val,Enc_in.detach().cpu().numpy()],axis = 0)
    b_all_val = Dec_in.detach().cpu().numpy() if len(b_all_val) == 0 else np.concatenate([b_all_val,Dec_in.detach().cpu().numpy()],axis = 0)
    c_all_val = Enc_mask.detach().cpu().numpy() if len(c_all_val) == 0 else np.concatenate([c_all_val,Enc_mask.detach().cpu().numpy()],axis = 0)
    d_all_val = Dec_mask.detach().cpu().numpy() if len(d_all_val) == 0 else np.concatenate([d_all_val,Dec_mask.detach().cpu().numpy()],axis = 0)
    out_all_val = Recur_data.detach().cpu().numpy() if len(out_all_val) == 0 else np.concatenate([out_all_val,Recur_data.detach().cpu().numpy()],axis = 0)
    out_mas_val = Recur_mask.detach().cpu().numpy() if len(out_mas_val) == 0 else np.concatenate([out_mas_val,Recur_mask.detach().cpu().numpy()],axis = 0)


In [ ]:
Pred_BART_model.eval()
D_all_val_test = []
# C_all_val = []
A_all_val_test = []
out_all_val_test = []
out_mas_val_test = []
c_all_val_test = []
for Enc_in, Enc_mask, Dec_in, Dec_mask, Recur_data, Recur_mask, drug_data, IDs in test_dataloader:
    len_test += Enc_in.shape[0]
    Enc_in = Enc_in.to(device) # N,10,26
    Dec_in = Dec_in.to(device) # N,10,26
    Enc_mask = Enc_mask.to(device) # N,10,1
    Dec_mask = Dec_mask.to(device) # N,10,1
    Recur_data= Recur_data.to(device)
    Recur_mask= Recur_mask.to(device)
    drug_data= drug_data.to(device)
    A,B,C,D = Pred_BART_model([Enc_in, Enc_mask, Enc_in, Enc_mask, drug_data], with_drug = True)
    A = torch.sigmoid(torch.clamp(A.squeeze(),max = 1e6, min = -1e6))
    A_all_val_test = A.detach().cpu().numpy() if len(A_all_val_test) == 0 else np.concatenate([A_all_val_test,A.detach().cpu().numpy()],axis = 0)
    D_all_val_test = D.detach().cpu().numpy() if len(D_all_val_test) == 0 else np.concatenate([D_all_val_test,D.detach().cpu().numpy()],axis = 0)
    c_all_val_test = Enc_mask.detach().cpu().numpy() if len(c_all_val_test) == 0 else np.concatenate([c_all_val_test,Enc_mask.detach().cpu().numpy()],axis = 0)
    out_all_val_test = Recur_data.detach().cpu().numpy() if len(out_all_val_test) == 0 else np.concatenate([out_all_val_test,Recur_data.detach().cpu().numpy()],axis = 0)
    out_mas_val_test = Recur_mask.detach().cpu().numpy() if len(out_mas_val_test) == 0 else np.concatenate([out_mas_val_test,Recur_mask.detach().cpu().numpy()],axis = 0)


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(out_all_val[:,1:].reshape(-1)[(out_mas_val[:,1:].reshape(-1)==1)], A_all_val.reshape(-1)[(out_mas_val[:,1:].reshape(-1)==1)])
AUC_train = roc_auc_score(out_all_val[:,1:].reshape(-1)[(out_mas_val[:,1:].reshape(-1)==1)], A_all_val.reshape(-1)[(out_mas_val[:,1:].reshape(-1)==1)])
fpr_test, tpr_test, thresholds_test = roc_curve(out_all_val_test[:,1:].reshape(-1)[(out_mas_val_test[:,1:].reshape(-1)==1)], A_all_val_test.reshape(-1)[(out_mas_val_test[:,1:].reshape(-1)==1)])
AUC_test = roc_auc_score(out_all_val_test[:,1:].reshape(-1)[(out_mas_val_test[:,1:].reshape(-1)==1)], A_all_val_test.reshape(-1)[(out_mas_val_test[:,1:].reshape(-1)==1)])
plt.plot(fpr, tpr,'b')
plt.plot(fpr_test, tpr_test,'r')
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')
plt.title("ROC curve")
plt.legend([f"Train : {AUC_train:.3f}",f"Test : {AUC_test:.3f}"])
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
plt.figure(figsize= (20,8))
for i in range(2,11):
    plt.subplot(2,5,i)
    fpr, tpr, thresholds = roc_curve(out_all_val[:,i].reshape(-1)[(out_mas_val[:,i].reshape(-1)==1)], A_all_val[:,i-1].reshape(-1)[(out_mas_val[:,i].reshape(-1)==1)])
    AUC_train = roc_auc_score(out_all_val[:,i].reshape(-1)[(out_mas_val[:,i].reshape(-1)==1)], A_all_val[:,i-1].reshape(-1)[(out_mas_val[:,i].reshape(-1)==1)])
    fpr_test, tpr_test, thresholds_test = roc_curve(out_all_val_test[:,i].reshape(-1)[(out_mas_val_test[:,i].reshape(-1)==1)], A_all_val_test[:,i-1].reshape(-1)[(out_mas_val_test[:,i].reshape(-1)==1)])
    AUC_test = roc_auc_score(out_all_val_test[:,i].reshape(-1)[(out_mas_val_test[:,i].reshape(-1)==1)], A_all_val_test[:,i-1].reshape(-1)[(out_mas_val_test[:,i].reshape(-1)==1)])
    plt.plot(fpr, tpr,'b')
    plt.plot(fpr_test, tpr_test,'r')
    plt.xlabel('FP Rate')
    plt.ylabel('TP Rate')
    plt.title(f"ROC curve | {i} year")
    plt.legend([f"Train : {AUC_train:.3f}",f"Test : {AUC_test:.3f}"])
plt.show()

In [ ]:
Last_weight = Pred_BART_model.decoder.fc_out_2.weight.unsqueeze(0).detach().cpu().numpy()

In [ ]:
D_all_val = D_all_val * Last_weight
D_all_val_test = D_all_val_test * Last_weight

In [ ]:
D_mean = np.mean(D_all_val, axis = 0)[np.newaxis,:,:]
D_std = np.std(D_all_val, axis = 0)[np.newaxis,:,:]
D_std[D_std == 0] = 1
D_all_val = (D_all_val-D_mean)/D_std
D_all_val_test = (D_all_val_test-D_mean)/D_std

In [ ]:

PCA_model = PCA(n_components=100)
PCA_result_train = PCA_model.fit_transform(D_all_val.reshape(-1,5120))
ssq = np.cumsum(PCA_model.explained_variance_ratio_)
lv = np.argmin(np.abs(ssq - 0.75)) + 1
lv=30
ssq_ = ssq[lv - 1]
PCA_result_train = PCA_result_train[:, :lv]
loading_pca = PCA_model.components_[:lv, :].T
print(f"LV: {lv} (% variance explained: {ssq_*100:.2f}%)")
PCA_result_test = PCA_model.transform(D_all_val_test.reshape(-1,5120))
PCA_result_test = PCA_result_test[:, :lv]

In [ ]:
ssq = np.cumsum(PCA_model.explained_variance_ratio_)
plt.figure(figsize=(6,3))
plt.plot(ssq,'*-')
plt.ylim(-0.1,1.1)
plt.xlim(-1,51)
plt.grid(1)

In [ ]:
ssq = (PCA_model.explained_variance_ratio_)
plt.figure(figsize=(6,3))
plt.plot(ssq,'*-')
# plt.ylim(-0.1,1.1)
plt.xlim(-1,51)
plt.grid(1)

In [ ]:
UMAP_model = UMAP()
UMAP_result = UMAP_model.fit_transform(PCA_result_train)
UMAP_result_test = UMAP_model.transform(PCA_result_test)

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
plt.scatter(UMAP_result[:,0],UMAP_result[:,1], s = 1)
plt.title('Trainset')
plt.subplot(1,2,2)
plt.scatter(UMAP_result_test[:,0],UMAP_result_test[:,1], s = 1)
plt.title('Validateset')
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
plt.scatter(UMAP_result[:,0],UMAP_result[:,1], s = 1, c = ['r' if i > 0 else 'b' for i in np.sum(out_all_val, axis = -1)])
plt.xlim(min(np.min(UMAP_result[:,0]),np.min(UMAP_result_test[:,0]))-0.5,max(np.max(UMAP_result[:,0]),np.max(UMAP_result_test[:,0]))+0.5)
plt.ylim(min(np.min(UMAP_result[:,1]),np.min(UMAP_result_test[:,1]))-0.5,max(np.max(UMAP_result[:,1]),np.max(UMAP_result_test[:,1]))+0.5)
plt.grid(1)
plt.subplot(1,2,2)
plt.scatter(UMAP_result_test[:,0],UMAP_result_test[:,1], s = 1, c = ['r' if i > 0 else 'b' for i in np.sum(out_all_val_test, axis = -1)])
plt.xlim(min(np.min(UMAP_result[:,0]),np.min(UMAP_result_test[:,0]))-0.5,max(np.max(UMAP_result[:,0]),np.max(UMAP_result_test[:,0]))+0.5)
plt.ylim(min(np.min(UMAP_result[:,1]),np.min(UMAP_result_test[:,1]))-0.5,max(np.max(UMAP_result[:,1]),np.max(UMAP_result_test[:,1]))+0.5)
plt.grid(1)


In [ ]:
from sklearn.cluster import KMeans
 
X = D_all_val.reshape(-1,5120)
 
kmeans = KMeans(n_clusters=12, random_state=0) ## KMeans 초기화
kmeans.fit(X) ## 클러스터링 알고리즘 수행
 
labels = kmeans.labels_ ## 클러스터링 결과

In [ ]:
l = []
for ID in ID_all:
    if ID in Table_2["new_ID"]:
        l += [Table_2[(Table_2["new_ID"] == ID)*(Table_2["count_idx"] == 0)].Test_date_normed.values[0]]
    else:
        l += [np.nan]

In [ ]:
Table_2["Cluster_num"] = ['' for _ in range(len(Table_2))]
for ID,clust in zip(ID_all,labels):
    if ID in Table_2.index:
        Table_2.loc[ID,["Cluster_num"]] = [clust for _ in range(len(Table_2.loc[[ID]]))]
Table_2    

In [ ]:
Table_2_sub[Table_2_sub.columns[:-3]].replace(0	, np.nan, inplace=True)
Table_2_sub


In [ ]:
Table_2_sub = Table_2.groupby(level = 0).count()
Table_2_sub.replace(0	, np.nan, inplace=True)
Table_2_sub["Data_count"] = [0 for _ in range(len(Table_2_sub))]
for n,ID in enumerate(ID_all):
    Table_2_sub.loc[ID,"Data_count"] = color_now[n]
Table_2_sub.groupby("Data_count").count()

In [ ]:
plt.figure(figsize=(15,3))
plt.imshow(((Table_2_sub.groupby("Data_count").count().values)/(Table_2_sub.groupby("Data_count").count()["new_ID"].values).reshape(11,1))[1:,2:])

In [ ]:
Table_2_sub.groupby("Data_count").sum()

In [ ]:
Table_case_1 = Table_2.iloc[[n for n,i in enumerate(Table_2["Cluster_num"].values) if i in [0,7] ]]
Table_case_1 = Table_case_1[Table_case_1.columns[1:]]
Table_case_1

In [ ]:
(Table_case_1[Table_case_1["count_idx"] == 0]["Cluster_num"] == 5).sum()

In [ ]:
Table_case_1 = Table_2.iloc[[n for n,i in enumerate(Table_2["Cluster_num"].values) if i in range(12) ]]

Table_case_1[Table_case_1.columns[-2:]]

In [ ]:
Table_2

In [ ]:
Table_2[Table_2["count_idx"]==0]["Cluster_num"]

In [ ]:
Table_2["count_idx"]

In [ ]:
sns.violinplot(data=Table_2[Table_2["count_idx"]==0], x="Cluster_num", y="Test_date_normed", order = [i for i in range(12)])

In [ ]:
plt.boxplot([Table_case_1[Table_case_1["Cluster_num"] ==0]['Test_date_normed'].values,Table_case_1[Table_case_1["Cluster_num"] ==7]['Test_date_normed'].values])

In [ ]:
plt.figure(figsize = (15,3))
plt.imshow(Table_case_1[Table_case_1["count_idx"] == 0].sort_values("Cluster_num").groupby("Cluster_num").count().values, vmax = 100)
plt.xticks([i*5 for i in range(27)])
plt.show()
plt.figure(figsize = (15,3))
plt.imshow(Table_case_1[Table_case_1["count_idx"] == 0].sort_values("Cluster_num").groupby("Cluster_num").mean().values)
plt.xticks([i*5 for i in range(27)])
plt.show()

In [ ]:
Table_case_1.columns[[24,28,30,33,39,42,45,63,75]]

In [ ]:
Table_case_1[Table_case_1["count_idx"] == 0].sort_values("Cluster_num").groupby("Cluster_num").mean()[['RDW']]

In [ ]:
Table_case_1[Table_case_1["count_idx"] == 0].sort_values("Cluster_num").groupby("Cluster_num").count()[['CRP (C-Reactive Protein)', 'Delta neutrophil', 'ESR', 'LUC(#)', 'MPC',
       'PDW(%)', 'PMN%', 'Free Fatty Acid', 'tCO2']]

In [ ]:
Table_case_1.columns[82]

In [ ]:
plt.figure(figsize = (5,15))
plt.imshow(((Table_case_1[Table_case_1["count_idx"] == 0].sort_values("Cluster_num")+10).fillna(0)).values[:,1:])
# plt.yticks([0,575,1047,1463],[0,575,1047,1463])
plt.show()

In [ ]:
Table_case_1[Table_case_1["count_idx"] == 0].sort_values("Cluster_num").fillna(0).values[2:]

In [ ]:
Table

In [ ]:
plt.scatter(UMAP_result[:,0],UMAP_result[:,1], s = 1, c = l)


In [ ]:
plt.figure(figsize = (12,4))
for i in range(12):
    plt.subplot(2,6,i+1)
    plt.scatter(UMAP_result[:,0],UMAP_result[:,1], s = 1, c = labels, alpha = [1 if j == i else 0.1 for j in labels])
    plt.title(f"cluster {i}")
plt.show()

In [ ]:
len(Table_2[["count_idx",cate]].groupby("count_idx").dropna())

In [ ]:
for cate in Table_2.columns[2:]:
    idx = 0
    if len(Table_2[["count_idx",cate]].groupby("count_idx").first().dropna()) == 1:
        plt.figure(figsize = (6,2))
        c_mean = []
        c_min = []
        c_max = []
        for cluster_idx in range(12):
            cluster_IDs = np.array(ID_all)[np.where(labels == cluster_idx)[0]]
            cluster_IDs = np.array(ID_all)[np.where(labels == cluster_idx)[0]]
            cluster_table = Table_2.iloc[[n for n,i in enumerate(Table_2["new_ID"].values) if i in cluster_IDs]][["count_idx",cate]]
            c_mean += [cluster_table.groupby("count_idx").mean().dropna().values[0,0]]
            c_min += [cluster_table.groupby("count_idx").min().dropna().values[0,0]]
            c_max += [cluster_table.groupby("count_idx").max().dropna().values[0,0]]
        plt.bar([i for i in range(12)], c_mean)
        plt.scatter([i for i in range(12)], c_min)
        plt.scatter([i for i in range(12)], c_max)
        plt.xlabel("clusters")
        plt.title(cate)
        plt.show()
        
    else:      
        plt.figure(figsize = (12,4))
        for cluster_idx in range(12):
            cluster_IDs = np.array(ID_all)[np.where(labels == cluster_idx)[0]]
            val_min = Table_2[cate].min()-0.5
            val_max = Table_2[cate].max()+0.5
            cluster_table = Table_2.iloc[[n for n,i in enumerate(Table_2["new_ID"].values) if i in cluster_IDs]][["count_idx",cate]]
            c_mean = cluster_table.groupby("count_idx").mean()
            c_min = cluster_table.groupby("count_idx").min()
            c_max = cluster_table.groupby("count_idx").max()
            for i in range(-1,9):
                if i not in c_mean.index:
                    c_mean.loc[i] = [np.nan for _ in range(c_mean.shape[1])]
                    c_min.loc[i] = [np.nan for _ in range(c_min.shape[1])]
                    c_max.loc[i] = [np.nan for _ in range(c_max.shape[1])]
            idx += 1
            plt.subplot(2,6,idx)
            plt.plot([i for i in range(-1,9)],c_mean.loc[[i for i in range(-1,9)]][cate])
            plt.xlim(-1.1,8.1)
            plt.ylim(val_min,val_max)
            plt.fill_between([i for i in range(-1,9)],c_min.loc[[i for i in range(-1,9)]][cate],c_max.loc[[i for i in range(-1,9)]][cate],alpha = 0.3)
        plt.suptitle(cate)
        plt.show()


In [ ]:
c_mean

In [ ]:
plt.scatter(UMAP_result[:,0],UMAP_result[:,1], s = 1, c = labels)
plt.colorbar()
plt.show()

In [ ]:
color_now = np.array([10-np.sum(c_all_val,axis = (1,2)) for _ in range(10)]).T.reshape(-1)[~c_all_val.reshape(-1)]
plt.figure(figsize = (15,10))
for idx in range(1,11):
    plt.subplot(3,4,idx)
    plt.scatter(UMAP_result[~c_all_val.reshape(-1) ,0],UMAP_result[~c_all_val.reshape(-1),1], alpha = [0.5 if i == idx else 0 for i in color_now], s = .5, c = color_now)
    plt.title(f"Data count : {idx}")
# plt.colorbar
plt.show()

In [ ]:
color_now = np.array([[i for i in range(10)] for _ in range(len(out_all_val))]).reshape(-1)[~c_all_val.reshape(-1)]
plt.figure(figsize = (15,10))
for idx in range(1,11):
    plt.subplot(3,4,idx)
    plt.scatter(UMAP_result[~c_all_val.reshape(-1) ,0],UMAP_result[~c_all_val.reshape(-1),1], alpha = [0.5 if i == (idx-1) else 0 for i in color_now], s = .5, c = color_now)
    plt.title(f"Timepoint (year after Op) : {idx}")
# plt.colorbar
plt.show()

In [ ]:
UMAP_result_D = UMAP_model.transform(D.detach().cpu().numpy().reshape(-1,512))


In [ ]:
Sample = train_dataloader.dataset.__getitem__(1387)
Sample["mask"]

In [ ]:
Enc_in = Sample["data"].unsqueeze(0).repeat(10,1,1).to(device) # 1,10,50
Enc_mask = torch.tril(torch.ones((10, 10)), diagonal = -1).T.unsqueeze(2).to(device).bool() # 10,10,1
Enc_in = Enc_in * (Enc_mask == 0)
Dec_in = torch.zeros_like(Enc_in).to(device) # 1,10,50
Dec_mask = torch.zeros_like(Enc_mask).to(device).bool() # 1,10,50
drug_data = Sample["Drug_data"].unsqueeze(0).repeat(10,1,1).to(device)
A,B,C,D = Pred_BART_model([Enc_in, Enc_mask, Dec_in, Dec_mask, drug_data], with_drug = True)

In [ ]:
D[0]

In [ ]:
Enc_mask[:,:,0]

In [ ]:
np.where(train_dataloader.dataset.ID_list=="P_003083")

In [ ]:
out_mas_val.shape

In [ ]:
Table_2 = pd.read_excel('230913_concat_table.xlsx')
Table_2.index = Table_2["new_ID"]

In [ ]:
plt.scatter(UMAP_result[color_now == 0,0],UMAP_result[color_now == 0,1], alpha = 0.3, s = 0.5, c = label_use[2])

In [ ]:
Table_2 = Table_2[Table_2["count_idx_2"]]

In [ ]:
Table_1["count_in_TF"]

In [ ]:
label_use

In [ ]:
label_use[i].unique()

In [ ]:
label_use = Table_2.loc[ID_all].groupby(level = 0).max()
plt.figure(figsize = (17,15))
idx = 0
for i in label_use:
    idx+=1
    plt.subplot(5,5,idx)
    try:
        if len(label_use[i].unique())<3:
            plt.scatter(UMAP_result[:,0],UMAP_result[:,1], alpha = [1 if j >0.5 else 0 for j in label_use[i]], s = 0.5, c = label_use[i])        
        else:
            plt.scatter(UMAP_result[:,0],UMAP_result[:,1], alpha = 0.3, s = 0.5, c = label_use[i])
        plt.colorbar()
        # plt.scatter(UMAP_result[:,0],UMAP_result[:,1], s = 1, c = label_use[i], alpha = 0.3)
    except:
        pass
    plt.title(i)
    if idx == 25:
        idx = 0
        plt.show()
        plt.figure(figsize = (15,15))
        

In [ ]:
plt.scatter(UMAP_result[: ,0],UMAP_result[:,1], c = l, s = 1, alpha =0.5)

In [ ]:
color_now = (10-np.sum(c_all_val,axis = (1,2))).T.reshape(-1)[:]
plt.figure(figsize = (15,10))
idx = 0
for i in range(10):
    idx+=1
    plt.subplot(3,4,idx)
    plt.scatter(UMAP_result[: ,0],UMAP_result[:,1], c = l, alpha = [1 if (j>i*0.5-3)*(j<=i*0.5-3+0.5) else 0 for j in l], s = 1)
# plt.colorbar

In [ ]:
Table_2

In [ ]:
ID_out = np.array(ID_all)[color_now == 1]
ID_out

In [ ]:
plt.boxplot([Table_2.loc[np.array(ID_all)[color_now == i]]["Test_date_normed"] for i in range(1,11)])
plt.xlabel('Data Count')
plt.ylabel('Relative Operation Date')
plt.xticks([i+1 for i in range(10)],[i+1 for i in range(10)])
plt.show()

In [ ]:
color_now = (10-np.sum(c_all_val,axis = (1,2))).T.reshape(-1)[:]
plt.figure(figsize = (15,10))
for idx in range(1,11):
    plt.subplot(3,4,idx)
    plt.scatter(UMAP_result[: ,0],UMAP_result[:,1], alpha = [0.5 if i == idx else 0 for i in color_now], s = 1)
    plt.title(f"Data count : {idx}")
# plt.colorbar

In [ ]:
color_now = ([i for i in range(10)]).reshape(-1)[~c_all_val.reshape(-1)]
plt.figure(figsize = (15,10))
for idx in range(1,11):
    plt.subplot(3,4,idx)
    plt.scatter(UMAP_result[: ,0],UMAP_result[:,1], alpha = [0.5 if i == (idx-1) else 0 for i in color_now], s = .5, c = color_now)
    plt.title(f"Timepoint (year after Op) : {idx}")
# plt.colorbar
plt.show()

In [ ]:
label_use = Table_1.loc[ID_all].groupby(level = 0).max()

In [ ]:
plt.scatter(UMAP_result_test[:,0],UMAP_result_test[:,1], s = 1, c = ['r' if i > 0 else 'b' for i in np.sum(out_all_val_test, axis = -1)])


In [ ]:
plt.scatter(UMAP_result[:,0],UMAP_result[:,1], s = 1, c = ['r' if i > 0 else 'b' for i in np.sum(out_all_val, axis = -1)])


In [ ]:
plt.scatter(PCA_result_test[:,0],PCA_result_test[:,1], s = 1, c = ['r' if i > 0 else 'b' for i in np.sum(out_all_val_test, axis = -1)])


In [ ]:
plt.scatter(PCA_result[:,0],PCA_result[:,1], s = 1, c = ['r' if i > 0 else 'b' for i in np.sum(out_all_val, axis = -1)])

In [ ]:
UMAP_model = umap.UMAP()
UMAP_result = UMAP_model.fit_transform(data['m'][label_NAN][:,using_z])

In [ ]:
plt.figure(figsize= (10,3))
idx = 0
idx+=1
plt.subplot(2,1,idx)
plt.hist(A_all_val.reshape(-1)[(out_all_val[:,1:].reshape(-1)==0)*(out_mas_val[:,1:].reshape(-1)==1)],color='r', alpha = 0.3,bins = [i/20 for i in range(21)])
plt.xlim(-0.1,1.1)
plt.legend(['Prediction for No Recur'])
idx+=1
plt.subplot(2,1,idx)
plt.hist(A_all_val.reshape(-1)[(out_all_val[:,1:].reshape(-1)==1)*(out_mas_val[:,1:].reshape(-1)==1)],color='b', alpha = 0.3,bins = [i/20 for i in range(21)])
plt.xlim(-0.1,1.1)
plt.legend(['Prediction for Recur'])
    # plt.scatter((A_all_val[:,i].reshape(-1)[out_mas_val[:,i].reshape(-1)==1]),
    #             (out_all_val[:,i].reshape(-1)[out_mas_val[:,i].reshape(-1)==1]),
    #             s = 1, alpha = 0.4)
    # plt.plot([-4,4],[-4,4],color = 'k')
# plt.xticks([-4,-2,0,2,4],[-4,-2,0,2,4])

#1 plt.ylabel('Answer')    
plt.xlabel('Output')

In [ ]:
plt.figure(figsize= (10,3))
idx = 0
idx+=1
plt.subplot(2,1,idx)
plt.hist(A_all_val_test.reshape(-1)[(out_all_val_test[:,1:].reshape(-1)==0)*(out_mas_val_test[:,1:].reshape(-1)==1)],color='r', alpha = 0.3,bins = [i/20 for i in range(21)])
plt.xlim(-0.1,1.1)
plt.legend(['Prediction for No Recur'])
idx+=1
plt.subplot(2,1,idx)
plt.hist(A_all_val_test.reshape(-1)[(out_all_val_test[:,1:].reshape(-1)==1)*(out_mas_val_test[:,1:].reshape(-1)==1)],color='b', alpha = 0.3,bins = [i/20 for i in range(21)])
plt.xlim(-0.1,1.1)
plt.legend(['Prediction for Recur'])
    # plt.scatter((A_all_val[:,i].reshape(-1)[out_mas_val[:,i].reshape(-1)==1]),
    #             (out_all_val[:,i].reshape(-1)[out_mas_val[:,i].reshape(-1)==1]),
    #             s = 1, alpha = 0.4)
    # plt.plot([-4,4],[-4,4],color = 'k')
# plt.xticks([-4,-2,0,2,4],[-4,-2,0,2,4])

#1 plt.ylabel('Answer')    
plt.xlabel('Output')

In [ ]:
from sklearn.decomposition import PCA
from umap import UMAP

In [ ]:
UMAP_model = umap.UMAP()
UMAP_result = UMAP_model.fit_transform(data['m'][label_NAN][:,using_z])
UMAP_result_test = UMAP_model.transform(test_data['m'][label_NAN_test][:,using_z])


In [ ]:
torch.sigmoid(torch.clamp(Pred_BART_model.decoder.fc_out_2(nn.ReLU()(Pred_BART_model.decoder.fc_out_1(C[0][idx]))).squeeze(),max = 1e6, min = -1e6))

In [ ]:
for idx in range(len(C)):
    # C = [i.detach().cpu().numpy() for i in C]
    i = 0
    mid_x = torch.sigmoid(torch.clamp(Pred_BART_model.decoder.fc_out_2(nn.ReLU()(Pred_BART_model.decoder.fc_out_1(C[i][idx]))).squeeze(),max = 1e6, min = -1e6))
    mid_x = mid_x.detach().cpu().numpy()
    plt.plot(mid_x)
plt.show()

In [ ]:
for idx in range(len(C[0])):
    # C = [i.detach().cpu().numpy() for i in C]
    plt.figure(figsize=(12,4))
    plt.subplot(2,1,1)
    for i in range(2):
        mid_x = torch.sigmoid(torch.clamp(Pred_BART_model.decoder.fc_out_2(nn.ReLU()(Pred_BART_model.decoder.fc_out_1(C[i][idx]))).squeeze(),max = 1e6, min = -1e6))
        mid_x = mid_x.detach().cpu().numpy()
        plt.plot(mid_x)
    plt.plot(Recur_data[idx,1:].detach().cpu().numpy())
    plt.plot(Recur_mask[idx,1:].detach().cpu().numpy())
    plt.legend(['In','Out','Ans','Mask'])
    plt.subplot(2,1,2)
    leg_sub = []
    for i in range(drug_data.shape[2]):
        if drug_data[idx,:,i].sum() > 0:
            plt.plot(drug_data[idx,:,i].detach().cpu().numpy())
            leg_sub += [Table_D.columns[i]]
    plt.legend(leg_sub)
    plt.show()

In [ ]:
A_all_val.shape

In [ ]:
(out_all_val==1).sum(axis = 0)

In [ ]:
(out_all_val==0).sum(axis = 0)

In [ ]:
plt.figure(figsize= (7,14))
idx = 0
for i in range(10):
    
    plt.subplot(10,2,idx//2*4+1+idx%2)
    plt.hist(A_all_val[:,i][(out_all_val[:,i+1]==0)*(out_mas_val[:,i+1]==1)],color='r', alpha = 0.3)
    plt.xlim(-0.1,1.1)
    # idx+=1
    # plt.subplot(5,4,idx)
    plt.subplot(10,2,idx//2*4+3+idx%2)
    plt.hist(A_all_val[:,i][(out_all_val[:,i+1]==1)*(out_mas_val[:,i+1]==1)],color='b', alpha = 0.3)
    plt.xlim(-0.1,1.1)
    plt.xticks([0,1],["No Recur", "Recur"])
    plt.title(f"{i+1} year")
    idx+=1
    # plt.scatter((A_all_val[:,i].reshape(-1)[out_mas_val[:,i].reshape(-1)==1]),
    #             (out_all_val[:,i].reshape(-1)[out_mas_val[:,i].reshape(-1)==1]),
    #             s = 1, alpha = 0.4)
    # plt.plot([-4,4],[-4,4],color = 'k')
# plt.xticks([-4,-2,0,2,4],[-4,-2,0,2,4])
# plt.ylabel('Answer')    
plt.xlabel('Output')

In [ ]:
plt.figure(figsize= (7,14))
idx = 0
for i in range(10):
    
    plt.subplot(10,2,idx//2*4+1+idx%2)
    plt.hist(A_all_val_test[:,i][(out_all_val_test[:,i+1]==0)*(out_mas_val_test[:,i+1]==1)],color='r', alpha = 0.3)
    plt.xlim(-0.1,1.1)
    # idx+=1
    # plt.subplot(5,4,idx)
    plt.subplot(10,2,idx//2*4+3+idx%2)
    plt.hist(A_all_val_test[:,i][(out_all_val_test[:,i+1]==1)*(out_mas_val_test[:,i+1]==1)],color='b', alpha = 0.3)
    plt.xlim(-0.1,1.1)
    plt.xticks([0,1],["No Recur", "Recur"])
    plt.title(f"{i+1} year")
    idx+=1
    # plt.scatter((A_all_val[:,i].reshape(-1)[out_mas_val[:,i].reshape(-1)==1]),
    #             (out_all_val[:,i].reshape(-1)[out_mas_val[:,i].reshape(-1)==1]),
    #             s = 1, alpha = 0.4)
    # plt.plot([-4,4],[-4,4],color = 'k')
# plt.xticks([-4,-2,0,2,4],[-4,-2,0,2,4])
# plt.ylabel('Answer')    
plt.xlabel('Output')

In [ ]:
idx = np.argmin(Enc_mask.sum(axis = [1,2]).detach().cpu().numpy())
for j in range(len(B)):
    print(j)
    for i in range(8):
        plt.subplot(2,4,i+1)
        plt.imshow(B[j][idx,i].detach().cpu().numpy(), vmin = 0, vmax = 1)
    plt.show()